In [4]:
# Imports - English comments
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from astroquery.jplhorizons import Horizons
# other imports as needed...
from astropy.time import Time
from astropy.coordinates import SkyCoord, CartesianRepresentation, ICRS, HeliocentricTrueEcliptic
import astropy.units as u

import time
from requests.exceptions import HTTPError


In [5]:
# fetch or load the data, e.g. df from JPL Horizons
# df = fetch_vectors_horizons('Voyager 1', '2012-01-01', '2025-12-01', n_points=150)
# For demo, use your existing df variables
def _call_horizons_with_retries(name, epochs_arg, max_retries=4, pause_base=1.0):
    """
    Helper: call Horizons(..., epochs=epochs_arg) with retries and exponential backoff.
    epochs_arg may be either:
      - a numpy array/list of JDs (short lists are OK), or
      - a dict {'start': 'YYYY-MM-DD', 'stop': 'YYYY-MM-DD', 'step': 'Nd'} which avoids long TLIST.
    Returns pandas DataFrame (or raises if all retries fail).
    """
    attempt = 0
    while attempt < max_retries:
        try:
            obj = Horizons(id=name, location='@sun', epochs=epochs_arg)
            table = obj.vectors()
            return table.to_pandas()
        except HTTPError as e:
            # If server returned 502/5xx, retry with backoff
            attempt += 1
            wait = pause_base * (2 ** (attempt - 1))
            print(f"HTTPError on attempt {attempt}/{max_retries}: {e}. Retrying after {wait:.1f}s...")
            time.sleep(wait)
        except Exception as e:
            # Other unexpected exceptions: raise immediately
            print("Unexpected error when querying Horizons:", type(e), e)
            raise
    # if we exit loop, all retries failed
    raise RuntimeError(f"Horizons query failed after {max_retries} attempts.")

def fetch_vectors_horizons(name, start_iso, stop_iso, n_points=200, prefer_range_threshold=250):
    """
    Robust fetch from JPL Horizons:
    - If n_points is small (< prefer_range_threshold), request explicit JD list.
    - If n_points is large, request a range (start/stop/step) to avoid long TLIST URIs.
    - Returns concatenated pandas DataFrame with x,y,z columns (AU).
    """
    t0 = Time(start_iso, format='iso', scale='utc')
    t1 = Time(stop_iso, format='iso', scale='utc')

    total_days = (t1 - t0).to(u.day).value
    if n_points <= 1:
        jd_grid = np.array([t0.jd])
    else:
        jd_grid = np.linspace(t0.jd, t1.jd, n_points)

    # If many points, prefer asking Horizons using a step-range (smaller URL)
    if n_points > prefer_range_threshold:
        # compute integer day step (at least 1 day)
        step_days = max(1, int(round(total_days / max(1, n_points - 1))))
        epochs_arg = {'start': start_iso, 'stop': stop_iso, 'step': f'{step_days}d'}
        print(f"Using range request to Horizons with step = {step_days} day(s) to avoid long URL.")
        df = _call_horizons_with_retries(name, epochs_arg)
        # Note: number of returned samples may differ slightly from n_points
        return df
    else:
        # For modest sized requests, use chunking to be safe (avoid giant single TLIST)
        max_chunk = 120  # keep each TLIST under ~120 entries
        dfs = []
        for i in range(0, len(jd_grid), max_chunk):
            chunk = jd_grid[i:i+max_chunk]
            print(f"Querying Horizons for {name}: chunk {i//max_chunk + 1}, {len(chunk)} epochs...")
            df_chunk = _call_horizons_with_retries(name, chunk)
            dfs.append(df_chunk)
        # concatenate and drop possible duplicate header rows
        df_all = pd.concat(dfs, ignore_index=True)
        # Some Horizons responses include overlapping rows; optionally drop exact-duplicate epochs
        if 'datetime_jd' in df_all.columns:
            df_all = df_all.drop_duplicates(subset=['datetime_jd'])
        return df_all

In [6]:
df_v1 = fetch_vectors_horizons('Voyager 1', '2012-01-01', '2025-12-01', 300)
# inspect
df_v1.head()


Using range request to Horizons with step = 17 day(s) to avoid long URL.


,targetname,datetime_jd,datetime_str,x,y,z,vx,vy,vz,lighttime,range,range_rate
0,Voyager 1 (spacecraft) (-31),2455927.5,A.D. 2012-Jan-01 00:00:00.0000,-25.715578,-94.346769,68.302156,-0.00121,-0.007916,0.005722,0.688906,119.280293,0.009799
1,Voyager 1 (spacecraft) (-31),2455944.5,A.D. 2012-Jan-18 00:00:00.0000,-25.736150,-94.481340,68.399435,-0.00121,-0.007916,0.005722,0.689868,119.446874,0.009799
2,Voyager 1 (spacecraft) (-31),2455961.5,A.D. 2012-Feb-04 00:00:00.0000,-25.756723,-94.615909,68.496712,-0.00121,-0.007916,0.005722,0.690830,119.613455,0.009799
3,Voyager 1 (spacecraft) (-31),2455978.5,A.D. 2012-Feb-21 00:00:00.0000,-25.777297,-94.750475,68.593985,-0.00121,-0.007916,0.005722,0.691792,119.780033,0.009799
4,Voyager 1 (spacecraft) (-31),2455995.5,A.D. 2012-Mar-09 00:00:00.0000,-25.797872,-94.885040,68.691255,-0.00121,-0.007916,0.005722,0.692754,119.946611,0.009799


In [7]:
# use the earlier plot_3d_trajectories code or quickly:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=df_v1['x'], y=df_v1['y'], z=df_v1['z'], mode='lines+markers', name='V1'))
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', marker=dict(size=6), name='Sun'))
fig.update_layout(scene=dict(aspectmode='auto'))
fig.show()
